In [0]:
%run ../configurations/paths_config

In [0]:
%sql
USE CATALOG vsarthicat;

In [0]:
from pyspark.sql.types import StringType,IntegerType,DoubleType,TimestampType,DateType,StructType,StructField
from pyspark.sql.functions import col,lit
from pyspark.sql import functions as sf

In [0]:
dbutils.widgets.text("p_file_date","2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
new_schema = StructType(fields=[StructField("raceId",IntegerType(),False),
                            StructField("year",IntegerType(),True),
                            StructField("round",IntegerType(),True),
                            StructField("circuitId",IntegerType(),True),
                            StructField("name",StringType(),True),
                            StructField("date",DateType(),True),
                            StructField("time",StringType(),True),
                            StructField("url",StringType(),True)])

In [0]:
raw_df = spark.read.option("header","true").schema(new_schema).csv(raw_path+f"/{v_file_date}/races.csv")

In [0]:
trimmed_df = raw_df.withColumn("race_timestamp",sf.expr("try_to_timestamp(concat(date, ' ', time), 'yyyy-MM-dd HH:mm:ss')")).drop("date","time","url")

In [0]:
renamed_df = trimmed_df.withColumnRenamed("raceId","race_id").withColumnRenamed("circuitId","circuit_id")

In [0]:
audited_df = renamed_df.withColumn("ingestion_time",sf.current_timestamp())

In [0]:
audited_df.write.format("delta").mode("overwrite").saveAsTable("vsarthicat.formula1_silver.races")